In [1]:
import math
import numpy as np
import torch
import gpytorch
import tqdm
import random
import time
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import sys
sys.path.append("../")
sys.path.append("../directionalvi/utils")
sys.path.append("../directionalvi")
import traditional_vi
from RBFKernelDirectionalGrad import RBFKernelDirectionalGrad
#from DirectionalGradVariationalStrategy import DirectionalGradVariationalStrategy
from dfree_directional_vi import train_gp, eval_gp
from metrics import MSE
import testfun
from csv_dataset import csv_dataset

In [2]:
dataset = csv_dataset("../experiments/real_data/WECs_DataSet/Perth_Data.csv", gradients=False, rescale=True)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
[48]


In [3]:
dataset.n

72000

In [4]:
# data parameters
n   = dataset.n
print("n is: ", n)
dim = dataset.dim
print("dims is: ", dim)

# training params
num_inducing = 500
num_directions = 1
minibatch_size = 500
num_epochs = 1000

# seed
#torch.random.manual_seed(0)
# use tqdm or just have print statements
tqdm = False
# use data to initialize inducing stuff
inducing_data_initialization = False
# use natural gradients and/or CIQ
use_ngd = False
use_ciq = False
num_contour_quadrature=15
# learning rate
learning_rate_hypers = 0.01
learning_rate_ngd    = 0.1
gamma  = 10.0
#levels = np.array([20,150,300])
#def lr_sched(epoch):
#  a = np.sum(levels > epoch)
#  return (1./gamma)**a
lr_sched = None

n is:  72000
dims is:  32


In [5]:
# train-test split
n_train = int(0.8*dataset.n)
n_test  = n - n_train
train_dataset,test_dataset = torch.utils.data.random_split(dataset,[n_train,n_test])

In [6]:
#loaders
train_loader = DataLoader(train_dataset, batch_size=minibatch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=n_test, shuffle=False)

In [7]:
test_y = [item[1] for item in test_loader]
test_x = [item[0] for item in test_loader]

# D-Free Grad SVGP

In [ ]:
# train
print("\n\n---DirectionalGradVGP---")
print(f"Start training with {n} trainig data of dim {dim}")
print(f"VI setups: {num_inducing} inducing points, {num_directions} inducing directions")
args={"verbose":True}
t1 = time.time()	
model,likelihood = train_gp(train_dataset,
                      num_inducing=num_inducing,
                      num_directions=num_directions,
                      minibatch_size = minibatch_size,
                      minibatch_dim = num_directions,
                      num_epochs =num_epochs, 
                      learning_rate_hypers=learning_rate_hypers,
                      learning_rate_ngd=learning_rate_ngd,
                      inducing_data_initialization=inducing_data_initialization,
                      use_ngd = use_ngd,
                      use_ciq = use_ciq,
                      lr_sched=lr_sched,
                      num_contour_quadrature=num_contour_quadrature,
                      tqdm=tqdm,**args
                      )
t2 = time.time()	

# save the model
# torch.save(model.state_dict(), "../data/test_dvi_basic.model")

# test
means, variances = eval_gp( test_dataset,model,likelihood,
                            num_directions=num_directions,
                            minibatch_size=n_test,
                            minibatch_dim=num_directions)
t3 = time.time()	




---DirectionalGradVGP---
Start training with 72000 trainig data of dim 32
VI setups: 500 inducing points, 1 inducing directions
All parameters to learn:
      variational_strategy.inducing_points
      torch.Size([500, 32])
      variational_strategy.inducing_directions
      torch.Size([500, 32])
      variational_strategy._variational_distribution.variational_mean
      torch.Size([1000])
      variational_strategy._variational_distribution.chol_variational_covar
      torch.Size([1000, 1000])
      mean_module.constant
      torch.Size([1])
      covar_module.raw_outputscale
      torch.Size([])
      covar_module.base_kernel.raw_lengthscale
      torch.Size([1, 1])
      noise_covar.raw_noise
      torch.Size([1])
Total number of parameters:  1033004.0
Epoch: 0; total_step: 0, loss: 2.4726170184510545, nll: 1.4002978832175332
Epoch: 0; total_step: 50, loss: 1.6684274296414636, nll: 1.1149669167810183
Epoch: 0; total_step: 100, loss: 1.519692141532555, nll: 0.969311184744317
Epoch

Epoch: 40; total_step: 4750, loss: 0.8516478631188985, nll: 0.3097374200872883
Epoch: 41; total_step: 4800, loss: 0.8677313477467139, nll: 0.2781649830288215
Epoch: 41; total_step: 4850, loss: 0.9447872040924618, nll: 0.3596252562954846
Epoch: 42; total_step: 4900, loss: 0.8088763859825773, nll: 0.2766123679133644
Epoch: 42; total_step: 4950, loss: 0.8837187102799768, nll: 0.2803226351912983
Epoch: 43; total_step: 5000, loss: 0.8433901753592635, nll: 0.3085149831603168
Epoch: 43; total_step: 5050, loss: 0.8712945137719028, nll: 0.3118054420102752
Epoch: 43; total_step: 5100, loss: 0.8455594809881213, nll: 0.27342867723215253
Epoch: 44; total_step: 5150, loss: 0.9103972407868193, nll: 0.3217839453970024
Epoch: 44; total_step: 5200, loss: 0.8638011398183396, nll: 0.27716025011179374
Epoch: 45; total_step: 5250, loss: 0.8382332114498079, nll: 0.28932614126557343
Epoch: 45; total_step: 5300, loss: 0.7572340909093768, nll: 0.23583008466728358
Epoch: 46; total_step: 5350, loss: 0.84075324855

Epoch: 85; total_step: 9950, loss: 0.8523967358979375, nll: 0.2536415667085113
Epoch: 86; total_step: 10000, loss: 0.853302373876237, nll: 0.3284592863857071
Epoch: 86; total_step: 10050, loss: 0.811946163628589, nll: 0.21461624231447973
Epoch: 87; total_step: 10100, loss: 0.7773121366464865, nll: 0.2236463775847393
Epoch: 87; total_step: 10150, loss: 0.8130405343034918, nll: 0.21906531091311968
Epoch: 87; total_step: 10200, loss: 0.7759241405364792, nll: 0.20947575661687656
Epoch: 88; total_step: 10250, loss: 0.7981430762255779, nll: 0.2536240875326203
Epoch: 88; total_step: 10300, loss: 0.8181125892419552, nll: 0.275349567955151
Epoch: 89; total_step: 10350, loss: 0.8302426739474054, nll: 0.26080929407524456
Epoch: 89; total_step: 10400, loss: 0.851077614999252, nll: 0.34253174493178296
Epoch: 90; total_step: 10450, loss: 0.8328980301112006, nll: 0.30087960203174374
Epoch: 90; total_step: 10500, loss: 0.7911886238206266, nll: 0.23800831793267233
Epoch: 90; total_step: 10550, loss: 0.

Epoch: 129; total_step: 15050, loss: 0.7641054196117827, nll: 0.21079819986759898
Epoch: 130; total_step: 15100, loss: 0.8292427252101013, nll: 0.24647499899812836
Epoch: 130; total_step: 15150, loss: 0.8100488353878404, nll: 0.19239013689975462
Epoch: 131; total_step: 15200, loss: 0.8630349022439685, nll: 0.3146752341561182
Epoch: 131; total_step: 15250, loss: 0.8424307309789354, nll: 0.2859826246151298
Epoch: 131; total_step: 15300, loss: 0.8438489311760863, nll: 0.23570624441565205
Epoch: 132; total_step: 15350, loss: 0.7797344257255401, nll: 0.19558110590714467
Epoch: 132; total_step: 15400, loss: 0.8653130906979016, nll: 0.29707713902511135
Epoch: 133; total_step: 15450, loss: 0.7652637882541641, nll: 0.20966841142259185
Epoch: 133; total_step: 15500, loss: 0.7764119318911562, nll: 0.2740288333989091
Epoch: 134; total_step: 15550, loss: 0.7944149182920607, nll: 0.1997265760535633
Epoch: 134; total_step: 15600, loss: 0.8293116938299597, nll: 0.22957455670746485
Epoch: 134; total_st

Epoch: 173; total_step: 20100, loss: 0.8335686425701087, nll: 0.2534799330913688
Epoch: 173; total_step: 20150, loss: 0.7912289062517456, nll: 0.1292317170072109
Epoch: 174; total_step: 20200, loss: 0.7985810945147466, nll: 0.16534455697294353
Epoch: 174; total_step: 20250, loss: 0.7505645452458573, nll: 0.18401425091642679
Epoch: 175; total_step: 20300, loss: 0.781302457012019, nll: 0.23019766001410624
Epoch: 175; total_step: 20350, loss: 0.8593973911819416, nll: 0.3322935820329917
Epoch: 175; total_step: 20400, loss: 0.7566075771151418, nll: 0.21302761545636786
Epoch: 176; total_step: 20450, loss: 0.8138316384668699, nll: 0.2668036459538772
Epoch: 176; total_step: 20500, loss: 0.7913485603067707, nll: 0.2369681869930946
Epoch: 177; total_step: 20550, loss: 0.8581391200937596, nll: 0.26485293410638405
Epoch: 177; total_step: 20600, loss: 0.7790153563196245, nll: 0.16830839019526575
Epoch: 178; total_step: 20650, loss: 0.7953798439825953, nll: 0.217942152487335
Epoch: 178; total_step: 

...

Epoch: 565; total_step: 65550, loss: 0.7986476016070507, nll: 0.2264926992597003
Epoch: 565; total_step: 65600, loss: 0.7756854926255115, nll: 0.17652216203552748
Epoch: 565; total_step: 65650, loss: 0.7830054722614713, nll: 0.1987757428605396
Epoch: 566; total_step: 65700, loss: 0.776198424617823, nll: 0.1793396504578351
Epoch: 566; total_step: 65750, loss: 0.8243773580690361, nll: 0.2814188258236563
Epoch: 567; total_step: 65800, loss: 0.7977709082393126, nll: 0.1952084115352752
Epoch: 567; total_step: 65850, loss: 0.7871796877755334, nll: 0.22389434510420897
Epoch: 568; total_step: 65900, loss: 0.7961422744628409, nll: 0.19437924875749377
Epoch: 568; total_step: 65950, loss: 0.820612341010376, nll: 0.20062943678301626
Epoch: 568; total_step: 66000, loss: 0.7583350022562554, nll: 0.24665579779339558
Epoch: 569; total_step: 66050, loss: 0.8087660216670849, nll: 0.20401041997080255
Epoch: 569; total_step: 66100, loss: 0.8104918433573118, nll: 0.1908243714119755
Epoch: 570; total_step: 

Epoch: 608; total_step: 70600, loss: 0.7940787618171098, nll: 0.196259639948438
Epoch: 609; total_step: 70650, loss: 0.7620438832668959, nll: 0.12815463223573004
Epoch: 609; total_step: 70700, loss: 0.7836643517905368, nll: 0.14570438028813215
Epoch: 609; total_step: 70750, loss: 0.7883055027848678, nll: 0.16185657792377406
Epoch: 610; total_step: 70800, loss: 0.7540612837625505, nll: 0.12561236157027292
Epoch: 610; total_step: 70850, loss: 0.7989138649694153, nll: 0.1709554751914655
Epoch: 611; total_step: 70900, loss: 0.8328507113551255, nll: 0.22490845225334857
Epoch: 611; total_step: 70950, loss: 0.7935897761374243, nll: 0.19781050508030854
Epoch: 612; total_step: 71000, loss: 0.7817408308929644, nll: 0.20548298897704642
Epoch: 612; total_step: 71050, loss: 0.7064971499672932, nll: 0.08945103680225623
Epoch: 612; total_step: 71100, loss: 0.843700600326663, nll: 0.2682036972791908
Epoch: 613; total_step: 71150, loss: 0.7880701295606138, nll: 0.25476825655735713
Epoch: 613; total_ste

Epoch: 652; total_step: 75650, loss: 0.7313274239553522, nll: 0.09007130720328134
Epoch: 652; total_step: 75700, loss: 0.7549152759460018, nll: 0.16388787858734855
Epoch: 653; total_step: 75750, loss: 0.7219279129864083, nll: 0.11180051999487746
Epoch: 653; total_step: 75800, loss: 0.7282364740224864, nll: 0.16213885712042164
Epoch: 653; total_step: 75850, loss: 0.7465024047793397, nll: 0.17601115791355829
Epoch: 654; total_step: 75900, loss: 0.8230812135823399, nll: 0.21153703154271755
Epoch: 654; total_step: 75950, loss: 0.8135901283438882, nll: 0.17544900810367514
Epoch: 655; total_step: 76000, loss: 0.8148161436023381, nll: 0.22992349240037913
Epoch: 655; total_step: 76050, loss: 0.8412586770330909, nll: 0.3133894045099062
Epoch: 656; total_step: 76100, loss: 0.7333354168623646, nll: 0.1491347867770174
Epoch: 656; total_step: 76150, loss: 0.7416331742045188, nll: 0.16168423191538467
Epoch: 656; total_step: 76200, loss: 0.7624383830685272, nll: 0.16172652117071354
Epoch: 657; total_

Epoch: 695; total_step: 80700, loss: 0.7883329567260794, nll: 0.24439953708341478
Epoch: 696; total_step: 80750, loss: 0.7155076542799552, nll: 0.10557568734576483
Epoch: 696; total_step: 80800, loss: 0.742918881906474, nll: 0.15886199614912788
Epoch: 696; total_step: 80850, loss: 0.7821268800267829, nll: 0.21080377604379852
Epoch: 697; total_step: 80900, loss: 0.8045722464434346, nll: 0.2093963905959435
Epoch: 697; total_step: 80950, loss: 0.7390630260682524, nll: 0.21712011017097718
Epoch: 698; total_step: 81000, loss: 0.7314178409047004, nll: 0.20143166139341023
Epoch: 698; total_step: 81050, loss: 0.7375807006260243, nll: 0.1360146118343663
Epoch: 699; total_step: 81100, loss: 0.7302770999391133, nll: 0.12311816927305544
Epoch: 699; total_step: 81150, loss: 0.7284111431172334, nll: 0.1144514151843915
Epoch: 700; total_step: 81200, loss: 0.7601882601035184, nll: 0.14464031719079332
Epoch: 700; total_step: 81250, loss: 0.752580774329305, nll: 0.18569495596388033
Epoch: 700; total_ste

Epoch: 739; total_step: 85750, loss: 0.6833456352756577, nll: 0.14832144859535026
Epoch: 739; total_step: 85800, loss: 0.7627584175243921, nll: 0.17860717297863965
Epoch: 740; total_step: 85850, loss: 0.7909493882294754, nll: 0.14588194352752798
Epoch: 740; total_step: 85900, loss: 0.814197315726615, nll: 0.18624455245162597
Epoch: 740; total_step: 85950, loss: 0.8165268570035364, nll: 0.1623888772115007
Epoch: 741; total_step: 86000, loss: 0.7577874787490576, nll: 0.1697329422914508
Epoch: 741; total_step: 86050, loss: 0.8096565631094277, nll: 0.24033181674401324
Epoch: 742; total_step: 86100, loss: 0.733207362333343, nll: 0.1162722377677498
Epoch: 742; total_step: 86150, loss: 0.7256479430430605, nll: 0.14551187165707616
Epoch: 743; total_step: 86200, loss: 0.7217327188540364, nll: 0.11483685195936956
Epoch: 743; total_step: 86250, loss: 0.7907326227143107, nll: 0.19718798631764173
Epoch: 743; total_step: 86300, loss: 0.7549604347303391, nll: 0.1879366805640543
Epoch: 744; total_step

In [ ]:

# compute MSE
#test_y = test_y.cpu()
test_mse = MSE(test_y[0],means)
# compute mean negative predictive density
test_nll = -torch.distributions.Normal(means, variances.sqrt()).log_prob(test_y[0]).mean()
print(f"At {n_test} testing points, MSE: {test_mse:.4e}, nll: {test_nll:.4e}.")
print(f"Training time: {(t2-t1):.2f} sec, testing time: {(t3-t2):.2f} sec")

#plot=1
#if plot == 1:
#    from mpl_toolkits.mplot3d import axes3d
#    import matplotlib.pyplot as plt
#    fig = plt.figure(figsize=(12,6))
#    ax = fig.add_subplot(111, projection='3d')
#    ax.scatter(test_x[0][:,0],test_x[:,1],test_y, color='k')
#    ax.scatter(test_x[0][:,0],test_x[:,1],means, color='b')
#    plt.title("f(x,y) variational fit; actual curve is black, variational is blue")
#    plt.show()

In [ ]:
# training params
#num_inducing = 50
#num_directions = 6
#minibatch_size = 200
#num_epochs = 100


# 2 directions
#At 104 testing points, MSE: 2.9133e+00, nll: 3.3945e+00. 
# 3 directions
#At 104 testing points, MSE: 2.9455e+00, nll: 3.3617e+00.
#Training time: 70.29 sec, testing time: 0.10 sec
# 4 directions
#At 104 testing points, MSE: 2.9810e+00, nll: 3.0743e+00.
#Training time: 57.68 sec, testing time: 0.08 sec
# 5 directions
#At 104 testing points, MSE: 2.9440e+00, nll: 3.6124e+00.
#Training time: 104.46 sec, testing time: 0.12 sec
# 6 directions
#At 104 testing points, MSE: 2.9795e+00, nll: 3.1092e+00.
#Training time: 127.73 sec, testing time: 0.10 sec
# 7 directions
#At 104 testing points, MSE: 2.9272e+00, nll: 3.6537e+00.
#Training time: 153.38 sec, testing time: 0.12 sec
# 8 directions
#At 104 testing points, MSE: 2.9503e+00, nll: 3.3300e+00.
#Training time: 173.86 sec, testing time: 0.15 sec
# 9 directions
# 10 directions

# Traditional SVGP

In [ ]:
model_t,likelihood_t = traditional_vi.train_gp(train_dataset,dim,
                                                   num_inducing=num_inducing,
                                                   minibatch_size=minibatch_size,
                                                   num_epochs=num_epochs,
                                                   use_ngd=use_ngd, use_ciq=use_ciq,
                                                   learning_rate_hypers=learning_rate_hypers,
                                                   learning_rate_ngd=learning_rate_ngd,
                                                   lr_sched=lr_sched,
                                                   num_contour_quadrature=num_contour_quadrature,gamma=gamma, verbose=True)

In [ ]:
means_t, variances_t = traditional_vi.eval_gp(test_dataset, model_t, likelihood_t, minibatch_size=n_test)

In [ ]:
# compute MSE
#test_y = test_y.cpu()
test_mse = MSE(test_y[0],means_t)
# compute mean negative predictive density
test_nll = -torch.distributions.Normal(means_t, variances_t.sqrt()).log_prob(test_y[0]).mean()
print(f"At {n_test} testing points, MSE: {test_mse:.4e}, nll: {test_nll:.4e}.")
print(f"Training time: {(t2-t1):.2f} sec, testing time: {(t3-t2):.2f} sec")

In [ ]:
# protein
# dfree and svgp
# At 9146 testing points, MSE: 5.9555e-01, nll: 1.1599e+00.
# At 9146 testing points, MSE: 6.2736e-01, nll: 1.1856e+00.

In [ ]:
#forest fire

# At 104 testing points, MSE: 3.0218e+00, nll: 4.2752e+00.
#Training time: 837.60 sec, testing time: 0.31 sec

#At 104 testing points, MSE: 2.8974e+00, nll: 3.5117e+00.
#Training time: 837.60 sec, testing time: 0.31 sec

In [ ]:
# At 14400 testing points, MSE: 1.0995e-01, nll: 3.0791e-01.
# At 14400 testing points, MSE: 1.2291e-01, nll: 3.9036e-01.